# 02 文档拆分

![image.png](imgs/RAG.png)
在上一节中，我们学会了如何加载各种格式的文档。本节的内容将对加载后的文档进行处理，对应上图的步骤3和4。加载文档后通常需要将它们做特定处理以适应大模型应用，例如将较长的文档拆分为合适模型上下文大小的块。

拆分文本时，一般不希望截断连贯的语义。`langchain`官方推荐的文本拆分器是`RecursiveCharacterTextSplitter`，它默认按照`[“\n\n”，“\n”，“”，“]`的列表拆分文本，这样做的效果是尽可能长时间地将所有段落（然后是句子，然后是单词）放在一起，因为这些段落通常看起来是语义相关最强的文本片段。

In [2]:
# 首先需要安装对应的库
pip install -qU langchain-text-splitters

SyntaxError: invalid syntax (4004064836.py, line 2)

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

with open('data/index.txt') as f:
    hetangyuese = f.read()

# 实例化一个拆分器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  # 每个块的最大字符数
    chunk_overlap=20,  # 块之间的重叠。块之间部分重叠以减少信息丢失
    length_function=len,  # 用来判定块大小的函数，这里按列表的长度来算
    is_separator_regex=False,  # 分隔符列表，默认为[“\n\n”，“\n”，“”，“]
)

texts = text_splitter.create_documents([hetangyuese])

print(type(texts))
print(type(texts[0]))
print(texts[0])
print(texts[1])

<class 'list'>
<class 'langchain_core.documents.base.Document'>
page_content='荷塘月色

作者: 朱自清'
page_content='这几天心里颇不宁静。今晚在院子里坐着乘凉，忽然想起日日走过的荷塘，在这满月的光里，总该另有一番样子吧。月亮渐渐地升高了，墙外马路上孩子们的欢笑，已经听不见了；妻在屋里拍着闰儿，迷迷糊糊地哼着眠歌'


观察输出，`RecursiveCharacterTextSplitter.create_documents()`返回一个列表，列表的每个元素是一个Document。虽然规定每个块的字符大小是100，但题目和作者与正文的第一段被精确的拆分了。

除了递归拆分器外，还有按照字符拆分的方法：

In [4]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=0
)
texts = text_splitter.split_text(hetangyuese)
print(texts[0])

荷塘月色

作者: 朱自清


token拆分器的方法。注意一些语言（比如中文）的字符可能编码为多个token。TokenTextSplitter可能将单个字符的多个token拆分为多个块，这会导致Unicode字符格式错误。应当使用RecursiveCharacterTextSplitter.from_tiktoken_encoder或CharacterTextSpliter.from_thiktoken_encoder避免Unicode格式错误。

In [5]:
from langchain_text_splitters import TokenTextSplitter

# 按token划分的类TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)
texts = text_splitter.split_text(hetangyuese)
print(texts[0])

# 建议使用RecursiveCharacterTextSplitter.from_tiktoken_encoder
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=100, chunk_overlap=0
)

texts = text_splitter.split_text(hetangyuese)
print(texts[0])

荷塘月色
荷塘月色

作者: 朱自清


`matedata`可以看做块的标签，为块添加`matedata`作为筛选和过滤的辅助信息。

In [6]:
metadatas = [{"document": 1}, {"document": 2}]
documents = text_splitter.create_documents([hetangyuese, hetangyuese], metadatas=metadatas)
print(documents[0])

page_content='荷塘月色

作者: 朱自清' metadata={'document': 1}


与`.create_documents()`不同，`.split_text()`返回的列表是纯文本的。

In [7]:
text_list = text_splitter.split_text(hetangyuese)[:2]
print(text_list)
len(text_list)

['荷塘月色\n\n作者: 朱自清', '这几天心里颇不宁静。今晚在院子里坐着乘凉，忽然想起日日走过的荷塘，在这满月的光里，总该另有一番样子吧。月亮渐渐地升高了，墙外马路上孩子们的欢笑']


2

针对程序语言，CodeTextSplitter支持拆分代码。

In [8]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)

# 支持的语言列表
print([e.value for e in Language])

# python语言的默认分割符
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)

['cpp', 'go', 'java', 'kotlin', 'js', 'ts', 'php', 'proto', 'python', 'rst', 'ruby', 'rust', 'scala', 'swift', 'markdown', 'latex', 'html', 'sol', 'csharp', 'cobol', 'c', 'lua', 'perl', 'haskell', 'elixir']


['\nclass ', '\ndef ', '\n\tdef ', '\n\n', '\n', ' ', '']

In [9]:
#下面是一个使用 PythonTextSplitter 的示例
PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Call the function
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(page_content='def hello_world():\n    print("Hello, World!")'),
 Document(page_content='# Call the function\nhello_world()')]

可以在官方文档查看更多的语言的分割[示例](https://python.langchain.com/v0.2/docs/how_to/code_splitter/)

以上的方法是基于规则分割的。下面介绍根据语义相似性进行划分的方法。拆分的逻辑是：如果embedding距离足够远，则拆分块。

In [10]:
import os
from langchain_community.embeddings import XinferenceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# 首先定义一个embedding模型(这里是基于xinference的)
server_url=os.environ.get("SERVER_URL")
model_uid=os.environ.get("EMBEDDING_MODEL_UID")
embeddings = XinferenceEmbeddings(
    server_url=server_url,
    model_uid=model_uid
)

text_splitter = SemanticChunker(embeddings)
docs = text_splitter.create_documents([hetangyuese])
print(docs[0].page_content)

荷塘月色

作者: 朱自清

　
　　这几天心里颇不宁静。今晚在院子里坐着乘凉，忽然想起日日走过的荷塘，在这满月的光里，总该另有一番样子吧。月亮渐渐地升高了，墙外马路上孩子们的欢笑，已经听不见了；妻在屋里拍着闰儿，迷迷糊糊地哼着眠歌。我悄悄地披了大衫，带上门出去。
　　沿着荷塘，是一条曲折的小煤屑路。这是一条幽僻的路；白天也少人走，夜晚更加寂寞。荷塘四面，长着许多树，蓊蓊郁郁的。路的一旁，是些杨柳，和一些不知道名字的树。没有月光的晚上，这路上阴森森的，有些怕人。今晚却很好，虽然月光也还是淡淡的。
　　路上只我一个人，背着手踱着。这一片天地好像是我的；我也像超出了平常的自己，到了另一世界里。我爱热闹，也爱冷静；爱群居，也爱独处。像今晚上，一个人在这苍茫的月下，什么都可以想，什么都可以不想，便觉是个自由的人。白天里一定要做的事，一定要说的话，现在都可不理。这是独处的妙处，我且受用这无边的荷香月色好了。曲曲折折的荷塘上面，弥望的是田田的叶子。叶子出水很高，像亭亭的舞女的裙。层层的叶子中间，零星地点缀着些白花，有袅娜地开着的，有羞涩地打着朵儿的；正如一粒粒的明珠，又如碧天里的星星，又如刚出浴的美人。微风过处，送来缕缕清香，仿佛远处高楼上渺茫的歌声似的。这时候叶子与花也有一丝的颤动，像闪电般，霎时传过荷塘的那边去了。叶子本是肩并肩密密地挨着，这便宛然有了一道凝碧的波痕。叶子底下是脉脉的流水，遮住了，不能见一些颜色；而叶子却更见风致了。
月光如流水一般，静静地泻在这一片叶子和花上。薄薄的青雾浮起在荷塘里。叶子和花仿佛在牛乳中洗过一样；又像笼着轻纱的梦。虽然是满月，天上却有一层淡淡的云，所以不能朗照；但我以为这恰是到了好处——酣眠固不可少，小睡也别有风味的。月光是隔了树照过来的，高处丛生的灌木，落下参差的斑驳的黑影，峭楞楞如鬼一般；弯弯的杨柳的稀疏的倩影，却又像是画在荷叶上。塘中的月色并不均匀；但光与影有着和谐的旋律，如梵婀玲上奏着的名曲。
　　荷塘的四面，远远近近，高高低低都是树，而杨柳最多。这些树将一片荷塘重重围住；只在小路一旁，漏着几段空隙，像是特为月光留下的。树色一例是阴阴的，乍看像一团烟雾；但杨柳的丰姿，便在烟雾里也辨得出。树梢上隐隐约约的是一带远山，只有些大意罢了。树缝里也漏着一两点路灯光，没精打采的，是渴睡人的眼。这时候最热闹的，要数树上的蝉声与水里的蛙声；但热闹是

In [11]:
from langchain.vectorstores import Chroma
# 定义一组文本文档
texts = [
    "Basquetball is a great sport.",
    "Fly me to the moon is one of my favourite songs.",
    "The Celtics are my favourite team.",
    "This is a document about the Boston Celtics",
    "I simply love going to the movies",
    "The Boston Celtics won the game by 20 points",
    "This is just a random text.",
    "Elden Ring is one of the best games in the last 15 years.",
    "L. Kornet is one of the best Celtics players.",
    "Larry Bird was an iconic NBA player.",
]  

# 使用这些文本文档和预训练的词嵌入创建一个检索器
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)

# 定义一个查询
query = "What can you tell me about the Celtics?"

# 基于这个查询检索相关的文档，并按相关性分数排序
docs = retriever.get_relevant_documents(query)
docs

e:\Anaconda\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='This is a document about the Boston Celtics'),
 Document(page_content='The Celtics are my favourite team.'),
 Document(page_content='The Boston Celtics won the game by 20 points'),
 Document(page_content='L. Kornet is one of the best Celtics players.'),
 Document(page_content='Basquetball is a great sport.'),
 Document(page_content='This is just a random text.'),
 Document(page_content='Fly me to the moon is one of my favourite songs.'),
 Document(page_content='Elden Ring is one of the best games in the last 15 years.'),
 Document(page_content='Larry Bird was an iconic NBA player.'),
 Document(page_content='I simply love going to the movies')]